In [6]:
import tensorflow as tf

## Data Prep

In [1]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-07-05 14:07:49--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  32.4MB/s    in 0.2s    

2023-07-05 14:07:50 (32.4 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [2]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [4]:
!head /content/dataset/fra.txt

Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)
Run!	Cours !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)
Run!	Courez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)
Run!	Prenez vos jambes à vos cous !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)
Run!	File !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)


## Preprocessing

In [8]:
text_dataset = tf.data.TextLineDataset("/content/dataset/fra.txt")
text_dataset

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
# find the longest sequence
for i in text_dataset.skip(190000):
    print(len(tf.strings.split(i, " ")))

In [11]:
VOCAB_SIZE = 20000
SEQUENCE_LENGTH = 64
EMBEDDING_DIM = 300

https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization?hl=en

Maps text features to integer sequences

In [13]:
english_vectorize_layer = tf.keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

In [14]:
french_vectorize_layer = tf.keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

In [15]:
def selector(input_text):
  split_text = tf.strings.split(input_text, '\t')
  return split_text[0:1], split_text[1:2]

In [16]:
split_dataset = text_dataset.map(selector)

In [17]:
for i in split_dataset.take(3):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va !'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche.'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'En route !'], dtype=object)>)


In [ ]:
english_training_data = split_dataset.map(lambda x,y : x) # input x,y and output x # only for english
english_vectorize_layer.adapt(english_training_data)

In [ ]:
french_training_data = split_dataset.map(lambda x,y : y) # input x,y and output x # only for english
french_vectorize_layer.adapt(french_training_data)

In [ ]:
def vectorizer(english, french):
  return english_vectorize_layer